In [105]:
from bs4 import BeautifulSoup, SoupStrainer
import requests
import urllib.request
import copy
import pandas as pd
from newspaper import Article

In [106]:
# выясняем сколько страниц на сайие
url="https://digital.gov.ru/ru/events/monitoring/?words=&type=&directions=&start_date=&end_date=&page=1"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0'}
result = requests.get(url, headers=headers)
# поиск по тегам
soup = BeautifulSoup(result.text, 'html.parser', parse_only=SoupStrainer('div'))
box = soup.find('div', class_='paging col-md-8 col-md-offset-4')
links = [link['href'] for link in box.find_all(href=True)]
num_page=''
for i in range (6, len(links[1])):
    if links[1][i]=='&':
        break
    else:
        num_page=num_page+links[1][i]
num_page=int(num_page)
num_page=1

In [107]:
# получить контент
def get_news(url):
    news = Article(url, language='ru')
    news.download()  # Загрузить веб-страницу
    news.parse()  # синтаксический анализ веб-страницы
    print('body: \ n', news.text)  # body content

In [109]:
# собираем сслки на статьи на странице каждой
dataframe=pd.DataFrame()
for k in range(1,num_page+1):
    url="https://digital.gov.ru/ru/events/monitoring/?words=&type=&directions=&start_date=&end_date=&page="+str(k)
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0'}
    result = requests.get(url, headers=headers)
    soup = BeautifulSoup(result.text, 'html.parser', parse_only=SoupStrainer('div'))
    box = soup.find_all('li', class_='')
    link=[]
    for i in range (0, len(box)):
        try:
            links = [link['href'] for link in box[i].find_all(href=True)]
            if links[0][0:11]=='/ru/events/' and len(links[0])>=11 and int(links[0][11]):
                get_news('https://digital.gov.ru'+links[0])
#                 link.append(copy.deepcopy(links[0]))
        except ValueError:
            e=1
#     print(link)

body: \ n МОСКВА, 22 мая. /ТАСС/. Предварительное внутрипартийное голосование "Единой России", как и дистанционное электронное голосование, проводится с применением российских технологий, санкции не помешали этому. Об этом заявил журналистам в воскресенье президент "Ростелекома" Михаил Осеевский.

"Надо отметить, что все технологии, которые используются и при создании федеральной системы дистанционного электронного голосования, и системы голосования, которую использует партия "Единая Россия", базируются на российских IT-решениях", - сказал он, отвечая на вопрос корреспондента ТАСС о возможном влиянии санкций на проведение праймериз "Единой России".

По словам главы "Ростелекома", это касается основы системы блокчейн, а также портала "Госуслуг". "Поэтому мы сегодня чувствуем себя абсолютно уверенными. Конечно санкции создают определенные сложности с приобретением серверов, но надо сказать, что благодаря активной поддержке президента и правительства за последнее время было сделано очень 

body: \ n Граждане России смогут сдать свои биометрические данные в Единую биометрическую систему (ЕБС) непосредственно в многофункциональных центрах (МФЦ) при помощи "Криптобиокабин", используемых для оформления загранпаспортов. Это позволит передавать данные в максимально защищенном режиме и избежать их утечек. Ранее сдать биометрические данные можно было в отделениях банков.

Минцифры подготовило проект постановления правительства, который обеспечивает возможность создания специализированных окон в МФЦ для сбора биометрических персональных данных физических лиц в ГИС ЕБС. Документ опубликован на портале нормативно-правовых актов.

Работа по оснащению МФЦ "Криптобиокабинами" проводилась с 2020 года и позволила передать функции по выдаче биометрических загранпаспортов нового поколения от МВД гражданскому персоналу. Срок завершения проекта несколько раз сдвигался из-за ограничений, вызванных пандемией, и технической сложности пусконаладочных работ. По данным минэкономразвития, на данны

body: \ n МОСКВА, 27 апреля. /ТАСС/. Около 1,5 млн подтвержденных учетных записей появилось на портале госуслуг с начала 2022 года, сообщается в среду в Telegram-канале Минцифры РФ.

"С начала 2022 года на "Госуслугах" появилось почти 1,5 млн подтвержденных учетных записей", - говорится в сообщении.

Учетную запись на "Госуслугах", как отмечается, можно подтвердить через интернет-банк, в МФЦ или в центрах обслуживания. "Личный кабинет на "Госуслугах" дает доступ к десяткам цифровых сервисов. С помощью портала можно оформить нужные документы, получить пособия и льготы, проверить и оплатить штрафы и задолженности", - напомнили в Минцифры.

ссылка на источник

Не допускается использование всех материалов, размещенных в разделе «Мониторинг СМИ» официального сайта Министерства связи и массовых коммуникаций РФ, без указания их правообладателя, указанного для каждой публикации


KeyboardInterrupt: 

In [ ]:
# headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0'}
# html_page = requests.get('https://digital.gov.ru/ru/events/41570/',headers=headers).content
# soup = BeautifulSoup(html_page, 'lxml')
# whitelist = ['h1', 'h2', 'h3','strong','b','u','em','i','ins','p','br','hr']
# out = ""

# for t in soup.find_all(text=True):
#     if t.parent.name in whitelist:
#         out += '{} '.format(t)

# escape = ['\r', '\n', '\t', '\xa0']
# for e in escape:
#     out = out.replace(e, '')
# out

In [112]:
data = pd.read_csv('vocab.csv')
data

,0
0,информация
1,система
2,информационный
3,защита
4,средство
...,...
995,актуальность
996,печать
997,однозначно
998,простой


In [121]:
url="https://digital.gov.ru/ru/events/41568/"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0'}
result = requests.get(url, headers=headers)
soup = BeautifulSoup(result.text, 'html.parser', parse_only=SoupStrainer('div'))
box = soup.find_all('div', class_='col-lg-8 col-md-8 text-content')
link=[]
for i in range (0, len(box)):
    print(box[i].find_all('p'))
    print()
#     links = [link['p'] for link in box[i].find_all('p')]

[<p style="margin-bottom: 25px"><img alt="" src="/uploaded/photos/rozhdenie-2.png" width="100%"/></p>, <p><strong>Москва, 20 мая 2022 года —</strong> Минцифры России совместно с Минздравом, Минюстом, Минфином и ФНС России существенно преобразовали процесс регистрации рождения ребенка.</p>, <p>Теперь это можно сделать полностью в электронном виде, пройдя следующие шаги:</p>, <p>У матери ребенка сохраняется возможность не оформлять электронный медицинский документ о рождении, ей предоставят его на бумажном носителе. С ним можно также подать заявление на Госуслугах или лично в органе ЗАГС или МФЦ. Таким образом, граждане, которые привыкли использовать бумажные документы, могут воспользоваться своим правом.</p>, <p>Для тех, кто выберет электронные документы, также сохраняется возможность получить в последующем бумажные документы. Для этого можно воспользоваться сервисом Госуслуг по выдаче повторных и первичных свидетельств и справок.</p>, <p> «Обычно ребенка ждут 9 месяцев, но этого ребенк

In [124]:
news = Article("https://digital.gov.ru/ru/events/41550/", language='ru')
news.download()  # Загрузить веб-страницу
news.parse()  # синтаксический анализ веб-страницы
print('body: \ n', news.text)  # body content
el=news.text

body: \ n Холдинг «Росэлектроника» (с 2009 года входит в состав государственной корпорации «Ростех») разработал беспроводные Bluetooth-датчики для умного дома и интернета вещей.

Об этом сообщает РИА Новости.

«В устройствах применяется технология Bluetooth с низким энергопотреблением (Bluetooth Low Energy, BLE). Это позволяет увеличить срок автономной работы изделий до десяти лет без замены батарей. IoT-датчики предназначены для измерения температуры, влажности, освещённости, магнитного поля. Информация с датчиков поступает в мобильное приложение и систему мониторинга», — отметили в госкорпорации.

«Ростех» подчеркнул, что датчики интернета вещей разработаны совместно с российской компанией «Тесла Смарт». Новые устройства изготовлены на базе герконов и отличаются от существующих аналогов расширенной сенсорикой и повышенной долговечностью.

Как сообщили в «Ростехе», изготовлением серийных выпусков займётся Рязанский завод металлокерамических приборов (РЗМКП, входит в холдинг «Росэлектр

In [125]:
el.replace("ссылка на источник", "")
el

'Холдинг «Росэлектроника» (с 2009 года входит в состав государственной корпорации «Ростех») разработал беспроводные Bluetooth-датчики для умного дома и интернета вещей.\n\nОб этом сообщает РИА Новости.\n\n«В устройствах применяется технология Bluetooth с низким энергопотреблением (Bluetooth Low Energy, BLE). Это позволяет увеличить срок автономной работы изделий до десяти лет без замены батарей. IoT-датчики предназначены для измерения температуры, влажности, освещённости, магнитного поля. Информация с датчиков поступает в мобильное приложение и систему мониторинга», — отметили в госкорпорации.\n\n«Ростех» подчеркнул, что датчики интернета вещей разработаны совместно с российской компанией «Тесла Смарт». Новые устройства изготовлены на базе герконов и отличаются от существующих аналогов расширенной сенсорикой и повышенной долговечностью.\n\nКак сообщили в «Ростехе», изготовлением серийных выпусков займётся Рязанский завод металлокерамических приборов (РЗМКП, входит в холдинг «Росэлектро